In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import matplotlib.image as mpimg
import numpy as np
import keras
## Run on google colab

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May 13 06:48:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
### NOTE: Please connect to GPU runtime on this Google Colab Notebook
os.environ['KAGGLE_USERNAME'] = " (your kaggle username )" # username from the json file
os.environ['KAGGLE_KEY'] = " (your kaggle key )" # key from the json file

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()
### TO RESET THE GPU

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!kaggle datasets download -d grantwiersum/marco-protein-crystal-image-recognition
### DOWNLOADING THE DATASET FROM THE KAGGLE WEBISTE,
## you may access many more gb of data from https://marco.ccr.buffalo.edu/data/test/ and https://marco.ccr.buffalo.edu/data/train/
## already in tfrecord format.

100% 2.05G/2.05G [00:28<00:00, 16.8MB/s]
100% 2.05G/2.05G [00:28<00:00, 76.3MB/s]


In [ ]:
!unzip -q /content/marco-protein-crystal-image-recognition.zip 

In [ ]:

### FUNCTION TO EXTRACT FEATURES FROM THE TFRECORDS FILE FORMAT

def _extract_fn(ds):
        # Extract features using the keys set during creation
        features = {
        "image/class/label": tf.io.FixedLenFeature([], tf.int64),
         "image/encoded":  tf.io.FixedLenFeature([],tf.string)
        }
        
        # Extract the data record
        sample = tf.io.parse_single_example(ds, features)
        image = tf.image.decode_jpeg(sample['image/encoded'],channels=3) 
        reshaped = tf.image.resize(image,[499,499],preserve_aspect_ratio=False)
        label = sample['image/class/label']
        if(label==0):
          lb=[1,0,0,0]
        elif(label==1):
          lb=[0,1,0,0] 
        elif(label==2):
          lb=[0,0,1,0]
        else:
          lb=[0,0,0,1]    
    
        return reshaped,lb     

In [ ]:

def flip_left_right(ds):
        features = {
          "image/class/label": tf.io.FixedLenFeature([], tf.int64),
          "image/encoded":  tf.io.FixedLenFeature([],tf.string)
          }
        seed = (2,3)
        # Extract the data record
        sample = tf.io.parse_single_example(ds, features)
        image = tf.image.decode_jpeg(sample['image/encoded'],channels=3) 
        reshaped = tf.image.resize(image,[499,499],preserve_aspect_ratio=False)
        label = sample['image/class/label']
        if(label==0):
          lb=[1,0,0,0]
        elif(label==1):
          lb=[0,1,0,0] 
        elif(label==2):
          lb=[0,0,1,0]
        else:
          lb=[0,0,0,1]
        reshaped = tf.image.stateless_random_flip_left_right(reshaped, seed);
        return  reshaped,lb   

In [ ]:
def flip_up_down(ds):
        features = {
          "image/class/label": tf.io.FixedLenFeature([], tf.int64),
          "image/encoded":  tf.io.FixedLenFeature([],tf.string)
          }
          
        # Extract the data record
        sample = tf.io.parse_single_example(ds, features)
        image = tf.image.decode_jpeg(sample['image/encoded'],channels=3) 
        reshaped = tf.image.resize(image,[499,499],preserve_aspect_ratio=False)
        seed=(2,3)
        label = sample['image/class/label']
        
        if(label==0):
          lb=[1,0,0,0]
        elif(label==1):
          lb=[0,1,0,0] 
        elif(label==2):
          lb=[0,0,1,0]
        else:
          lb=[0,0,0,1]    
       
        reshaped = tf.image.stateless_random_flip_left_right(reshaped, seed);
         
        return reshaped, lb

In [ ]:
BATCH_SIZE=16
AUTOTUNE = tf.data.experimental.AUTOTUNE
### FUNCTION TO CREATE DATASET
def create_dataset(file_list,shuffle_buffer_size=1000):
    ds = tf.data.TFRecordDataset(file_list)
    ds = ds.shuffle(buffer_size=len(file_list))
    ds1 = ds.map(flip_up_down,num_parallel_calls=AUTOTUNE)
    ds2 = ds.map(flip_left_right,num_parallel_calls=AUTOTUNE)
    ds = ds.map(_extract_fn,num_parallel_calls=AUTOTUNE)
    ds = ds.concatenate(ds1)
    ds = ds.concatenate(ds2)
    ds= ds.batch(BATCH_SIZE)  ### SPLITTING THE DATA INTO BATCHES
    ds = ds.prefetch(buffer_size=AUTOTUNE) ### PREFETCH WILL FEED THE EXAMPLES TO MODEL ONE EXAMPLE AT A TIME, MORE MEMORY EFFICIENT AND CONVENIENT
    return ds

In [ ]:
filepaths = ['/content/train-00001-of-00407','/content/train-00002-of-00407','/content/train-00003-of-00407','/content/train-00004-of-00407','/content/train-00005-of-00407','/content/train-00006-of-00407','/content/train-00007-of-00407','/content/train-00008-of-00407','/content/train-00009-of-00407','/content/train-00010-of-00407','/content/train-00011-of-00407','/content/train-00012-of-00407','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/train-00027-of-00407','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/train-00028-of-00407','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/train-00029-of-00407','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/train-00030-of-00407','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/train-00031-of-00407','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/train-00032-of-00407','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/train-00033-of-00407','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/train-00013-of-00407']
train_ds = create_dataset(filepaths)  ### TRAIN DATASET

In [ ]:
# del train_ds

In [ ]:
count1=0
count2=0
count3=0
count4=0
seed = (2,3)
index =0;
ds1 = tf.TensorArray(dtype=tf.float32,size=0,dynamic_size=True)
for sample in train_ds:
    label = sample[1]
    if(label[0]):
      count1+=1;
    elif(label[1]):
      count2+=1;
    elif(label[2]):
      count3+=1;
    elif(label[3]):
      count4+=1;      
print("Number of examples for \n");
print("Class 1:"+str(count1)+"Class 2:"+str(count2)+"Class 3:"+str(count3)+"Class 4:"+str(count4))

In [ ]:
test_filepaths = ['test-00001-of-00046','test-00002-of-00046','test-00003-of-00046','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/test-00010-of-00046','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/test-00012-of-00046','/content/drive/MyDrive/Study Project 2019A7PS0097U(Giri)/test-00013-of-00046']
test_ds = create_dataset(test_filepaths)## TEST DATASET


In [ ]:
#def show_any_image(index):
#    msg ="Type is:"
#    if y_train[index]==0:
#        msg+="Clear"
#    elif y_train[index]==1:
#        msg+="Crystal"
#    elif y_train[index]==2:
#        msg+="Precipitate"
#    else:
#        msg+="Others"
#    
#    plt.imshow(x_train[index])
#    plt.title(msg)

In [ ]:

# Create a callback that saves the model's weights so that it can be accessed later, because the model is not permanent on google colab notebooks
# saving the model frequently so that progress is not lost
checkpoint_path = "/content/drive/MyDrive/Project/checkpoints/"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


In [ ]:
### To track the model training accuracy live graph (accuracy, loss , etc)
import datetime
log_dir = "/content/drive/MyDrive/Project/logs/"
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)


# Training on the InceptionV3 model
### from the imagenet competition

In [ ]:
## didnt get as good accuracy
from tensorflow.keras.applications.inception_v3 import InceptionV3  
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Input
input_tensor = Input(shape=(499, 499, 3))
# create the base pre-trained model"
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False,input_tensor=input_tensor)

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.layers import AveragePooling2D,Flatten,Dropout
from tensorflow.keras.optimizers import Adam
headModel = base_model.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dense(4, activation="softmax")(headModel)


In [ ]:
model = Model(inputs=base_model.input, outputs=headModel)

In [ ]:
model.summary()

In [ ]:
for layer in base_model.layers:
	layer.trainable = True

In [ ]:
del train_ds

In [ ]:
opt = Adam(lr=3e-2)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt,metrics=['categorical_accuracy'])

In [ ]:
%load_ext tensorboard

log_dir = "/content/drive/MyDrive/Project/logs/train/"
%tensorboard --logdir "/content/drive/MyDrive/Project/logs/exp2/"
### TO VIEW GRAPHS FOR WHILE TRAINING THE DATA (ACCURACY , LOSS)

In [ ]:
model.fit(train_ds,verbose=1,epochs=5,callbacks=[tensorboard_callback,cp_callback],validation_data=test_ds)
model.save('/content/drive/MyDrive/Models/InceptionV3/') ## got maximum of 74% accuracy


# Training on Xception model
 this model got better accuracy

In [ ]:
## performed relatively better
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Input

input_tensor2 = Input(shape=(499, 499, 3))
input_tensor2 = tf.keras.applications.xception.preprocess_input(input_tensor2)
base_model2 = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=input_tensor2,
    input_shape=(499,499,3)
)

83689472/83683744 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.layers import AveragePooling2D,Flatten,Dropout
from tensorflow.keras.optimizers import Adam
headModel2 = base_model2.output
headModel2 = AveragePooling2D(pool_size=(4, 4))(headModel2)
headModel2 = Flatten(name="flatten")(headModel2)
headModel2 = Dense(128, activation="relu")(headModel2)
headModel2 = Dense(128, activation="relu")(headModel2)
headModel2 = Dense(4, activation="softmax")(headModel2)

In [ ]:
model2 = Model(inputs=base_model2.input, outputs=headModel2)

In [ ]:
for layer in base_model2.layers:
	layer.trainable = True
    

In [ ]:
opt = Adam(lr=2e-2)
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt,metrics=['categorical_accuracy'])

In [ ]:
## TO LOAD THE MODEL
model2= tf.keras.models.load_model('/content/drive/MyDrive/Project/Xception/')
### DOWNLOAD LINK FOR THE TRAINED MODEL: https://drive.google.com/drive/folders/194ySuLOc4qloytkw0uMhHYebadmKjZnx?usp=sharing
## use this as base for further training

In [ ]:
#MODEL TRAINING
model2.fit(train_ds,verbose=1,epochs=2,callbacks=[tensorboard_callback,cp_callback ],validation_data=test_ds) 
#Maximum accuracy: 83%
## t

Epoch 1/2
      1/Unknown - 8s 8s/step - loss: 0.3883 - categorical_accuracy: 0.8750

ResourceExhaustedError: ignored

In [ ]:
##TO SAVE THE MODEL
model2.save('/content/drive/MyDrive/Project/Xception/')  

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Study Project resources/Xception/assets
